In [8]:
import mesa
from mesa.space import MultiGrid, PropertyLayer

In [44]:
class Model(mesa.Model):
    def __init__(self, size: int, n_ants: int, ant_vision = 1):
        super().__init__()
        self.n_ants = n_ants
        food_layer = PropertyLayer(name="food", width = size, height= size, default_value=0, dtype=int)
        pheromone_layer = PropertyLayer(name="pheromone", width = size, height= size, default_value=0, dtype=int)
        self.grid = MultiGrid(width=size, height=size, torus=True, 
                              property_layers=[food_layer, pheromone_layer])

        anthill_pos = (size//2, size//2)
        ants = Ant.create_agents(self, n_ants)
        n_foodpile = 3
        food = FoodPile.create_agents(self, n = n_foodpile )


        #place ants
        for agent in ants:
            self.grid.place_agent(agent, anthill_pos)

        #place food
        x = self.rng.integers(0, self.grid.width // 4, size=(n_foodpile,))
        y = self.rng.integers(0, self.grid.height // 4, size=(n_foodpile,))
        # for a, i, j in zip(agents, x, y):
        #     # Add the agent to a random grid cell
        #     self.grid.place_agent(a, (i, j))

         

    def step(self):
        for agent_class in self.agent_types:
            self.agents_by_type[agent_class].shuffle_do("step")

def dist(point1, point2):
    #manhattan distance
    return abs(point1[0] - point2[0]) + abs(point1[1] - point2[1])
    
class Ant(mesa.Agent):
    def __init__(self, model,carrying_capacity =1):
        super().__init__(model)
        #TODO @property
        self.food_bag = 0
        self.carrying_capacity = carrying_capacity
        #self.model = model

    def step(self):
        if self.food_bag == 0:
            self.explore()
        elif self.pos == self.model.anthill_pos:
            self.food_bag -=1
            print(f"Ant {self.unique_id} returned food!")
        else:
            self.return_to_anthill()

    def return_to_anthill(self):
        #we assume ants remember the way back to the anthill
        #compute the closet cell to the anthill
   

        #which neighboring cell is closest to anthill?
        neighbor_distances = {}
        for neighbor_cell in self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False):
            neighbor_distances[neighbor_cell] = dist(neighbor_cell, self.grid.anthill_pos)
        #if there is a tie, choose a random one
        move_candidates = [k for k, v in neighbor_distances.items() if v == min(neighbor_distances.values())]
        if len(move_candidates) > 1:
            move = self.rng.choice(move_candidates)
        else:
            move = move_candidates[0]
        self.model.grid.move_agent(self, move)



    def explore(self):
        # Move to a neighboring cell
        for neighbor_cell in self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False):
            food_cells = [neighbor for neighbor in self.model.grid.get_cell_list_contents(neighbor_cell) if isinstance(neighbor, FoodPile)]
            pheromone_cells = [neighbor for neighbor in self.model.grid.get_cell_list_contents(neighbor_cell) if isinstance(neighbor, Pheromone)]
            if food_cells and self.carrying_capacity > self.food_bag:
                # Pick up food
                food = food_cells[0]
                food.n_food -= 1
                self.food_bag += 1
                self.model.grid.move_agent(self, neighbor_cell)
                print(f"Ant {self.unique_id} picked up food!")

            elif pheromone_cells:
                # Follow pheromone trail
                #pick strongest pheromone
                move = self.rng.choice(pheromone_cells)
                self.model.grid.move_agent(self, move)

        print(self.pos)
        #             return
        print("I'm an ant")



    @classmethod
    def create_agents(cls, model, n: int):
        return [cls(model) for _ in range(n)]
# class Pheromone(mesa.Agent, decay="linear"):
#     def __init__(self, model, half_life: int):
#         super().__init__(model)
#         self.half_life = half_life
#     @property
#     def time_to_live(self):
#         self.time_to_live = half_life*2
#         #self.model = model
#     def decay(self):
#         if self.decay == "linear":
#             self.time_to_live -= 1

#         pass
#     def step(self):
#         print("I'm a pheromone")

class FoodPile(mesa.Agent):
    def __init__(self, model, n_food: int = 30):
        super().__init__(model)
        self.n_food = n_food
        #self.model = model

    def step(self):
        if self.n_food < 1:
            self.model.grid.remove_agent(self)

In [45]:
m = Model(size=10, n_ants=3)
m.step()

(5, 5)
I'm an ant
(5, 5)
I'm an ant
(5, 5)
I'm an ant
I'm food
I'm food
I'm food


In [52]:
d = {1: 2, 2: 2, 3: 4, 4: 3}
#get all keys with minimum value
print()
print(min(d, key=lambda x: x[1]))

[1, 2]
